In [4]:
# from faqer.services.classificator.eval import NavecDistCalculator, IOWDistCalculator, RDTModel
from faqer.services.data.utils import get_lines, get_text, tokenize_text

In [5]:
# iow_calc = IOWDistCalculator()
# nav_calc = NavecDistCalculator()
# rdt_calc = RDTModel()